<a href="https://colab.research.google.com/github/hamzajg/movies-rec-colab/blob/master/MoviesRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 3.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678227 sha256=4b96d090104b3462bf1e6d1dbd9fc5638e591b634f61e3cf59f2a504294e3ce4
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [4]:
!git clone https://github.com/hamzajg/movies-rec-colab.git

Cloning into 'movies-rec-colab'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 65 (delta 26), reused 29 (delta 7), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [0]:
%cd movies-rec-colab/

/content/movies-rec-colab


In [0]:
!git pull

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 5), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/hamzajg/movies-rec-colab
   4440613..0dc3bcb  master     -> origin/master
Updating 4440613..0dc3bcb
Fast-forward
 MoviesRec.ipynb       | 226 ++++++++++++++++++++++++++++++++++++++++++++++++++
 RecommenderMetrics.py | 154 ++++++++++++++++++++++++++++++++++
 2 files changed, 380 insertions(+)
 create mode 100644 MoviesRec.ipynb
 create mode 100644 RecommenderMetrics.py


In [11]:
%cd /content/

/content


In [6]:
import sys
sys.path.insert(0, 'movies-rec-colab')

from MovieLens import MovieLens

ml = MovieLens()
print('Loading Movies Rating..')
data = ml.loadMovieLensLatestSmall()

Loading Movies Rating..


In [7]:
print('Computing movie popularity ranks so we can measure novelty later..')
rankings = ml.getPopularityRanks()

Computing movie popularity ranks so we can measure novelty later..


In [8]:
from surprise import KNNBaseline
print('Computing item similarities so we can measure diversity later ...')
fullTrainSet = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
simsAlgo = KNNBaseline(sim_options=sim_options)
simsAlgo.fit(fullTrainSet)

Computing item similarities so we can measure diversity later ...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [9]:
from surprise.model_selection import train_test_split
from surprise import SVD
print('Building recommendation models..')
trainSet, testSet = train_test_split(data, test_size=.25, random_state=1)
algo = SVD(random_state=10)
algo.fit(trainSet)

Building recommendation models..


In [12]:
print('Computing recommendations ..')
predictions = algo.test(testSet)
from RecommenderMetrics import RecommenderMetrics
print('Evaluating accuracy of model ..')
print('RMSE:', RecommenderMetrics.RMSE(predictions))
print('MAE:', RecommenderMetrics.MAE(predictions))

Computing recommendations ..
Evaluating accuracy of model ..
RMSE: 0.9033701087151801
MAE: 0.6977882196132263


In [17]:
print('Evaluating Top-10 Recommendation ..')
from surprise.model_selection import LeaveOneOut
# Set aside on rating per user for testing
LOOCV = LeaveOneOut(n_splits=1, random_state=1)

for trainSet, testSet in LOOCV.split(data):
  print('Computing recommmendation with leaveOneOut..')
  # Traim model without left-out ratings
  algo.fit(trainSet)
  # Predicts ratings fro left-out ratings only
  leftOutPredictions = algo.test(testSet)
  # Build predictions fro all ratings not in the training set
  print('Predict All Missing Ratings..')
  bigTestSet = trainSet.build_anti_testset()
  allPredictions = algo.test(bigTestSet)

  print('Compte top-10 rating per user')
  topNPredicted = RecommenderMetrics.GetTopN(allPredictions, n=10)
  print('Hit Rate: ', RecommenderMetrics.HitRate(topNPredicted, leftOutPredictions))
  # Break down hit rate by rating value
  print("\nrHR (Hit Rate by Rating value): ")
  RecommenderMetrics.RatingHitRate(topNPredicted, leftOutPredictions)
  print('cHR: Rating >= 4: ', RecommenderMetrics.CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))
  print('ARHR: ', RecommenderMetrics.AverageReciprocalHitRank(topNPredicted, leftOutPredictions))


Evaluating Top-10 Recommendation ..
Computing recommmendation with leaveOneOut..
Predict All Missing Ratings..
Compte top-10 rating per user
Hit Rate:  0.029806259314456036

rHR (Hit Rate by Rating value): 
3.5 0.017241379310344827
4.0 0.0425531914893617
4.5 0.020833333333333332
5.0 0.06802721088435375
cHR: Rating >= 4:  0.04960835509138381
ARHR:  0.0111560570576964


In [21]:
print('Computing complete recommendation, no hold outs...')
algo.fit(fullTrainSet)

Computing complete recommendation, no hold outs...


In [22]:
print('Test The Model...')
bigTestSet = fullTrainSet.build_anti_testset()
allPredictions = algo.test(bigTestSet)
topNPredictions = RecommenderMetrics.GetTopN(allPredictions, n=10)

Test The Model...


In [23]:
# Print user coverage with a minimum predicted rating of 4.0:
print("\nUser coverage: ", RecommenderMetrics.UserCoverage(topNPredicted, fullTrainSet.n_users, ratingThreshold=4.0))

# Measure diversity of recommendations:
print("\nDiversity: ", RecommenderMetrics.Diversity(topNPredicted, simsAlgo))

# Measure novelty (average popularity rank of recommendations):
print("\nNovelty (average popularity rank): ", RecommenderMetrics.Novelty(topNPredicted, rankings))


User coverage:  0.9523099850968704
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Diversity:  0.9685719430531259

Novelty (average popularity rank):  508.73897360222986


In [25]:
#exporting the model

from surprise import dump
import os

print("exporting the model")
file_name = os.path.expanduser('./model.pkl')
dump.dump(file_name, algo=algo)
print("file dumped")

exporting the model
file dumped


In [26]:
_, loaded_algo = dump.load(os.path.expanduser('model.pkl'))
print("file loaded")

file loaded


In [32]:
trainset = data.build_full_trainset()
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
recs = RecommenderMetrics.GetTopN(predictions_loaded_algo,10)[11]
response_list = {'source':{"id":"SVD_model"},'movieIds':[],'predictedRatings':[]}
i = 0
while (len(response_list['predictedRatings']) < 10 and i < len(recs)):
    response = {
            'movieId':recs[i][0],
            'predicted rating':recs[i][1]
    }
    response_list['predictedRatings'].append(response)
    response_list['movieIds'].append(recs[i][0])
    i = i+1
print(response_list)

{'source': {'id': 'SVD_model'}, 'movieIds': [923, 50, 296, 48516, 1201, 778, 79132, 26614, 97938, 80489], 'predictedRatings': [{'movieId': 923, 'predicted rating': 4.786690147626509}, {'movieId': 50, 'predicted rating': 4.663960836500775}, {'movieId': 296, 'predicted rating': 4.629207742358713}, {'movieId': 48516, 'predicted rating': 4.626710956183247}, {'movieId': 1201, 'predicted rating': 4.473962936131099}, {'movieId': 778, 'predicted rating': 4.4602680144326925}, {'movieId': 79132, 'predicted rating': 4.4484307062801545}, {'movieId': 26614, 'predicted rating': 4.438853917810938}, {'movieId': 97938, 'predicted rating': 4.428651822454953}, {'movieId': 80489, 'predicted rating': 4.383122923457444}]}
